In [10]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '90'
customer_name='%wbmason%'
buckets = """Multiple Buckets"""
strategy_version_input=682
attribute_id_input=2425
formatted_attribute = 'Length'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

14156


14156

In [12]:
df['buckets'].explode().value_counts()[0:15]

Labels, Stickers & Tickets                                    5214
Mailing & Packing Boxes                                       3818
Protective Packing Materials                                  1685
Envelopes/Mailers                                             1100
Rugs & Mats                                                    444
Fixings & Fasteners                                            414
Papers                                                         262
Liners & Sheeting                                              212
Tie-Downs                                                      153
Tape Dispensers                                                152
Boxes, Baskets, & Bins                                         140
Shipping & Packaging Tools, Dispensers & Replacement Parts      93
Furniture Accessories & Replacement Parts                       89
Shipping & Packaging Products Variety Packs                     71
Shredded & Tissue Papers                                      

In [29]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*)|()'
height['matches']=height['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
total_na=height[height['matches'].astype(str)=='[]']
regex_pattern_total=r'(?i)(\d+\"?.?\s?x)|()'
total_na['new_matches']=total_na['long_desc'].apply(lambda x: re_extract(regex_pattern_total, x))
total_na_values=total_na[total_na['new_matches'].astype(str)=='[]']
print(len(total_na_values))
total_na_values[curation_col]='n/a'

804


# General

In [13]:

# regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\sH)|(\b\d+.?\"\sH)|(\b\d+\.\d*.?\"\sH)|(\d*\s?\-\d*\/\d*\\\"\sH)|(\d*\s\-?\d*\/\d*\\\"\sH)|(\d+\.?\d*''\sH)|()'
# # regex_pattern_general=r'(?i)(\b\d+\.\d*.?\"\s?H)|(\b\d+.?\"\s?H)|(\b\d+\.\d*.?\"\s?H)|(\d*\s?\-\d*\/\d*\\\"\s?H)|(\d*\s\-?\d*\/\d*\\\"\s?H)|(\d+\.?\d*''\s?H)|()'
# height['matches'] = height['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['name_matches'] = height['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
# height['matches']= height['matches'].apply(lambda x: sorted(x, key=natural_key))
# print(len(height))
# print(len(height[height['matches'].astype(str)=='[]']))
# print(len(height[height['matches'].astype(str)!='[]']))

In [10]:
# # 3
# (\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|

# # All
# (?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")

In [11]:
labels=height[height['buckets'].astype(str)=='Labels, Stickers & Tickets']
labels.head(1)

product_id external_id  \
2525   525242098  5637518765   

                                                               product_name  \
2525  W.B. Mason Auto Supplies Slogan, Buy Here Pay Here, Yellow/Red, 12/PK   

                                                                                                                                                                                 long_desc  \
2525  Slogan, Buy Here Pay Here, Yellow/Red, 12/PK•Great way to highlight vehicle details•Removable vinyl•Durable and weatherproof•Easy on and easy off•Slit on back for easy installation   

      customer_id attributes                     buckets  bucket_id values  
2525           90       None  Labels, Stickers & Tickets       8612   None

In [85]:
tubes_new=labels
ft=r'(?i)(\d+.?\d*\/?\d*.?\s?x\s?\d+.?\d*\/?\d*\s?ft)|()'              
tubes_new['matches']=tubes_new['product_name'].apply(lambda x: re_extract(ft, x))
tubes_new=tubes_new[tubes_new['matches'].astype(str)=='[]']
regex_pattern_double=r'(?i)(\d+.?\d*\/?\d*.?\s?x\s?\d+.?\d*\/?\d*)|()'              
tubes_new['matches']=tubes_new['product_name'].apply(lambda x: re_extract(regex_pattern_double, x))
tubes_new=tubes_new[tubes_new['matches'].astype(str)!='[]']
len(tubes_new)

2969

In [86]:
tubes_new['matches']=tubes_new['matches'].str.get(0).apply(lambda x: x.lower())
tubes_new[['width','height']]=tubes_new['matches'].str.split("x",expand=True,)
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'7 200','7',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
tubes_new['height']=tubes_new['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
tubes_new['height']=tubes_new['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
tubes_new['height_new'] = tubes_new['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

In [87]:
# tubes_new[0:500]

In [88]:
tubes_new[tubes_new['height_new'].astype(str)=='4 1']

product_id external_id  \
6971    434484305  5637463038   
13257   434484283  5637463015   

                                           product_name  \
6971      THERMAL TRANSFER LABEL 4.0X4 1 ACROSS 1500/RL   
13257   DIRECT THERMAL LABEL 4 X4 1"CORE 750/RL 8RLS/CS   

                                                long_desc  customer_id  \
6971     THERMAL TRANSFER LABEL 4.0X4 1 ACROSS 1500/RL• -           90   
13257  DIRECT THERMAL LABEL 4 X4 1"CORE 750/RL 8RLS/CS• -           90   

      attributes                     buckets  bucket_id values  matches width  \
6971        None  Labels, Stickers & Tickets       8612   None  4.0x4 1   4.0   
13257       None  Labels, Stickers & Tickets       8612   None   4 x4 1    4    

      height height_new  
6971     4 1        4 1  
13257    4 1        4 1

In [89]:
# tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'9 34','0.375',str(measure)))

In [90]:
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

In [94]:
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'1 500/','1',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'1110/','11',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'11310/','1',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'8 250/','8',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'3730/','3',str(measure)))

tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'2 1','2',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'3 1','3',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'1 1','1',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'6430','6',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'5 1','5',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'6 1','6',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'4 1','4',str(measure)))
tubes_new['height_new']=tubes_new['height_new'].apply(lambda measure: re.sub(r'8 1','1',str(measure)))

In [95]:
tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_frac(x))
tubes_new['height_new']=tubes_new['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
tubes_new[curation_col]=tubes_new['height_new']
tubes_new[curation_col]=tubes_new[curation_col].apply(lambda x: str(x)+' in')

# Protective Packing Materials    

In [11]:
packing=height[height['buckets'].astype(str)=='Protective Packing Materials']
len(packing)#variable

NameError: name 'height' is not defined

In [12]:

# not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# not_tube['length']=not_tube['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# not_tube['width']=not_tube['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# not_tube['length']=not_tube['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# not_tube['width']=not_tube['width'].apply(lambda quote: quote.replace('"', ''))

# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# not_tube['length_new']=not_tube['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# not_tube['width_new']=not_tube['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_fraction(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_fraction(x))

# not_tube['length_new'] = not_tube['length_new'].apply(lambda x: remove_frac(x))
# not_tube['width_new'] = not_tube['width_new'].apply(lambda x: remove_frac(x))

# not_tube['length_new']=not_tube['length_new'].astype(float)
# not_tube['width_new']=not_tube['width_new'].astype(float)

# not_tube['Max'] = not_tube[['length_new', 'width_new']].values.max(1)
# not_tube['Min'] = not_tube[['length_new', 'width_new']].values.min(1)


# TWOPLACES = Decimal(10) ** -2
# not_tube['Max']=not_tube['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x)))
# not_tube['Min']=not_tube['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

# envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
# envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())


# envelope[['length', 'width']]=envelope['envelope_matches'].str.split("x",expand=True,)

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

# envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
# envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

# envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
# envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))
# # envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
# # envelope['width']=envelope['width'].apply(lambda quote: quote.replace('-',''))


# regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
# envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
# envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)


# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

# envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
# envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

# Mailing & Packing Boxes

In [99]:
Boxes=height
Boxes=Boxes[Boxes['buckets'].astype(str)=='Mailing & Packing Boxes']
regex_pattern_tube=r'(?i)(tube)|()'
tubes=Boxes
tubes['tube_matches']=tubes['long_desc'].apply(lambda x: re_extract(regex_pattern_tube, x))

not_tube=tubes[tubes['tube_matches'].astype(str)=='[]']
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
not_tube['matches']=not_tube['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
not_tube=not_tube[not_tube['matches'].astype(str)!='[]']

In [100]:
not_tube['matches']=not_tube['matches'].str.get(0).apply(lambda x: x.lower())
not_tube[['length', 'width', 'height']]=not_tube['matches'].str.split("x",expand=True,)

In [101]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1232','12',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'15250','15',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'24141680','2',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'16200','16',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'9275','9',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'1832','18',str(measure)))

In [102]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))

In [103]:
not_tube['height']=not_tube['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
not_tube['height']=not_tube['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [104]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
not_tube['height_new']=not_tube['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
not_tube['height_new'] = not_tube['height_new'].apply(lambda x: remove_fraction(x))

In [105]:
not_tube['height_new'] = not_tube['height_new'].apply(lambda x: remove_frac(x))
not_tube['height_new']=not_tube['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
not_tube['height_new']=not_tube['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [106]:
not_tube[curation_col]=not_tube['height_new']
not_tube[curation_col]=not_tube[curation_col].apply(lambda x: str(x)+' in')

In [107]:
print(len(not_tube))

2814


In [108]:
not_tube

product_id external_id  \
0       704220980  5637549910   
1       704220978  5637549908   
2       704220956  5637550007   
3       675184647  5637543518   
4       675184645  5637543516   
...           ...         ...   
14051   434483446  5637462487   
14053   434482642  5637462655   
14054   434482640  5637462653   
14055   434482628  5637462638   
14147   434492586  5637464360   

                                                                          product_name  \
0              W.B. Mason Co. Fixed-Depth Shipping boxes, 12" x 10" x 6", Brown, 25/BD   
1               W.B. Mason Co. Fixed-Depth Shipping boxes, 10" x 8" x 6", Brown, 25/BL   
2      W.B. Mason Co. Brown Corrugated - Multi-Depth boxes, 24L x 16W x 12H, 20/Bundle   
3                         W.B. Mason Co. Corrugated boxes, 10" x 9" x 9", Kraft, 25/BD   
4                         W.B. Mason Co. Corrugated boxes, 10" x 9" x 6", Kraft, 25/BD   
...                                                                                ...   
14051                                        CORR BOX TOP 42 3/8 X 5X2 1/2 275C 113161   
14053                                          CORR BOX RSC 31X25X18 350DW PLAIN 78843   
14054                                             CORR BOX RSC 12-3/4X 11-1/4X13 275DW   
14055                                    CORRUGATED BOX-18X13X7 1/2 RSC/GLUE 32ECT J18   
14147                                    CORRUGATED BOX 22X18X16 RSC 32ECT NPI MEDICAL   

                                                                                                                                                                                                                                                                                                                  long_desc  \
0                                                                                   Fixed-Depth Shipping boxes, 12" x 10" x 6", Brown, 25/BD•This listing is for a bundle of 25 boxes•Strong, 200 lb. corrugated Kraft•ECT-32 C-Flute - holds up to 65 lbs.•Shipped flat and assembles with shipping tape (sold separately)   
1                                                                                    Fixed-Depth Shipping boxes, 10" x 8" x 6", Brown, 25/BL•This listing is for a bundle of 25 boxes•Strong, 200 lb. corrugated Kraft•ECT-32 C-Flute - holds up to 65 lbs.•Shipped flat and assembles with shipping tape (sold separately)   
2      Brown Corrugated - Multi-Depth boxes, 24L x 16W x 12H, 20/Bundle•This listing is for a bundle of 20 boxes•Box sides are scored at multiple depths to allow easy height adjustment for the perfect fit•No carton sizer required, just cut down to the desired size•Manufactured from 200 lb. corrugated Kraft; ECT-32   
3                                                    Corrugated boxes, 10" x 9" x 9", Kraft, 25/BD•This listing is for a bundle of 25 boxes•Quality standard strength industrial Corrugated Boxes•Manufactured from 200#/ECT-32 kraft corrugated•Corrugated boxes are recyclable•Sold and shipped flat in bundle quantities   
4                                                    Corrugated boxes, 10" x 9" x 6", Kraft, 25/BD•This listing is for a bundle of 25 boxes•Quality standard strength industrial Corrugated Boxes•Manufactured from 200#/ECT-32 kraft corrugated•Corrugated boxes are recyclable•Sold and shipped flat in bundle quantities   
...                                                                                                                                                                                                                                                                                                                     ...   
14051                                                                                                                                                                                                                                                                          CORR BOX TOP 42 3/8 X 5X2

# Envelope

In [16]:
envelope=height[height['buckets'].astype(str)=='Envelopes/Mailers']
regex_pattern_general=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*)|(\d+\.?\d*.?\s?x\s?\d+\.?\d*.?)|()'
envelope['envelope_matches']=envelope['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope_na=envelope[envelope['envelope_matches'].astype(str)=='[]']

In [22]:
envelope_na[curation_col]='n/a'

In [131]:
envelope=height[height['buckets'].astype(str)=='Envelopes/Mailers']
regex_pattern_general=r'(?i)(roll)|()'
envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)=='[]']

In [132]:
regex_pattern=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|()'              
envelope['matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern, x))
envelope=envelope[envelope['matches'].astype(str)=='[]']

In [133]:
# envelope[0:500] # Waiting on Arv and Ross

In [134]:
regex_pattern_general=r'(?i)(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\s?\-?\d+\/\d+)|(\d+\.\d*.?\s?x\s?\d+\.\d*)|(\d+\s?\-?\d+\/\d+.?\s?x\s?\d+\.?\d*)|(\d+\s?.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+)|(\d+.?\d*.?\s?x\s?\d+.?\d*)|(\d+\.?\s?\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|()'    
envelope['envelope_matches']=envelope['product_name'].apply(lambda x: re_extract(regex_pattern_general, x))
envelope=envelope[envelope['envelope_matches'].astype(str)!='[]']

In [135]:
# def array(df):
#     df['matches_one']=df['envelope_matches'].apply(lambda x : np.array(x)[0])
#     try:
#         df['matches_two']=df['envelope_matches'].apply(lambda x : np.array(x)[1])
#     except:
#         df['matches_two']='n/a'

In [136]:
# def drop_characters(data):
#     data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))

In [137]:
# array(envelope)

In [138]:
# envelope['match_one'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [139]:
# try:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[1])
# except:
#     envelope['matches'][4:6]=envelope['envelope_matches'][4:6].apply(lambda x : np.array(x)[0])

In [140]:
# double=envelope[envelope['external_id'].astype(str)=='5637503643']
# double=envelope[0:1]
# double

In [141]:
# double['n_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[0])
# # double['m_matches']=double['envelope_matches'].apply(lambda x : np.array(x)[1])
# double

In [142]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# double= double.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [143]:
# double=double.apply(lambda x: f'"{x}"')
# double=double.apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# double[['one','two']]=double['envelope_matches'].str.split(",",expand=True,)
# double=double.apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# double

In [144]:
envelope['envelope_matches']=envelope['envelope_matches'].str.get(0)
envelope['envelope_matches']=envelope['envelope_matches'].apply(lambda x: x.lower())

In [147]:
envelope

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches]
Index: []

In [129]:
envelope[['width', 'length', 'height']]=envelope['envelope_matches'].str.split("x",expand=True,)

ValueError: Columns must be same length as key

In [126]:
envelope

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, envelope_matches, matches]
Index: []

In [127]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'77-34','34',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'12 500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 2500\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12 1000\/','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))

envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'12\,100','12',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'10 100/','10',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'16 100','16',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'19 500','19',str(measure)))

KeyError: 'length'

In [128]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))

envelope['length']=envelope['length'].apply(lambda measure: re.sub(r"\'|,",'',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure:   re.sub(r"\'|,",'',str(measure)))

KeyError: 'length'

In [103]:
envelope['length']=envelope['length'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
envelope['width']=envelope['width'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))

envelope['length']=envelope['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
envelope['width']=envelope['width'].apply(lambda quote: quote.replace('"', ''))

regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
envelope['length_new']=envelope['length'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
envelope['width_new']=envelope['width'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)

In [104]:
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'12 1000','12',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'12100','12',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'16 100','16',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'12 2500\/','12',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'19 500','19',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'10 100','10',str(measure)))

envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'12\/','12',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'8 100/','8',str(measure)))
envelope['length_new']=envelope['length_new'].apply(lambda measure: re.sub(r'10/','10',str(measure)))

In [105]:
envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_fraction(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_fraction(x))

envelope['length_new'] = envelope['length_new'].apply(lambda x: remove_frac(x))
envelope['width_new'] = envelope['width_new'].apply(lambda x: remove_frac(x))

In [106]:
TWOPLACES = Decimal(10) ** -2
envelope['length_new']=envelope['length_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
envelope[curation_col]=envelope['length_new']
envelope[curation_col]=envelope[curation_col].apply(lambda x: str(x)+' in')

In [107]:
len(envelope)

5509

In [108]:
envelope

product_id external_id  \
14      434500384  5637480100   
15      434500366  5637480082   
113     434466076  5637435550   
130     434514519  5637497374   
131     434515977  5637497369   
...           ...         ...   
19647   434519029  5637499423   
19648   434508729  5637490547   
19651   434508747  5637490565   
19652   496166081  5637509320   
19656   434518692  5637501271   

                                                                                       product_name  \
14                              W.B. Mason Co. Bubble Lined Poly Mailers, 7 1/4" x 8", White, 25/CS   
15                          Sealed Air Jiffy� Rigi Bag� Mailers, 1, 7 1/4" x 10 1/2", Kraft, 250/CS   
113                        Polyair Polyair XPAK Poly Bubble Mailers, #5, 10.5� x 16�, White, 100/CS   
130                    JAM Paper A6 Passport Invitation Envelopes, 4 3/4" x 6 1/2", Granite, 250/BX   
131    JAM Paper A9 Metallic Envelopes, 5 3/4" x 8 3/4", Silver Pearlized Elegance Stardream, 50/BX   
...                                                                                             ...   
19647      JAM Paper A6 Metallic Invitation Envelopes, 4 3/4" x 6 1/2", Vista Blue Stardream, 25/PK   
19648    JAM Paper Policy Business Colored Envelopes, #10, 4 1/8" x 9 1/2", Fuchsia Hot Pink, 25/PK   
19651             JAM Paper Policy Colored Envelopes, #11, 4 1/2" x 10 3/8", Yellow Recycled, 25/PK   
19652       JAM Paper A7 Strathmore Invitation Envelopes, 5 1/4" x 7 1/4", Bright White Laid, 50/BX   
19656                               Lindenmeyr Explorer Envelopes, 4 1/8" x 9 1/2", 24 lb., 2500/CT   

                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
14                                                                                                                                                                                            Bubble Lined Poly Mailers, 7 1/4" x 8", White, 25/CS•Mailers combine the moisture resistance of a polyethylene bag with the cushioning bubble lining•Light-weight for lower shipping costs•Tamper and tear resistant•Self-seal   
15                                                                                                                                                                                                                                        Jiffy� Rigi Bag� Mailers, 1, 7 1/4" x 10 1/2", Kraft, 250/CS•Laminated kraft fiberboard construction•Extra-rigid•Self-adhesive closure and easy-open tear strip•Top loading design   
113    Polyair XPAK Poly Bubble Mailers, #5, 10.5� x 16�, White, 100/CS•100% polyethylene has strong weather and puncture resistance•Exterior film has opaque gray lining on inside, preventing seeing through and protecting privacy•Hot melt self-seal closure helps ensure a tamper-evident seal and content security•High slip lining allows easier product insertion and extraction•1/3 the weight of macerated mailers   
130                                                                                                                                                                                                         A6 Passport Invitation Envelopes, 4 3/4" x 6 1/2", Granite, 250/BX•Measures 4 3/4" x 6 1/2"•Available in multiple bold hues•Sturdily made from high quality paper•Contains a straight flap with firm gum closure   
131                                                                                                           A9 Metallic Envelopes, 5 3/4" x 8 3/4", Silver Pearlized Elegance Stardream, 50/BX•Measures 5 3/4" x 8 3/4"•Well-crafted from thick, high quality paper•Has a stylish monarch flap with

# Office Chairs

In [34]:
# oc=height[height['buckets'].astype(str)=='Office Chairs']
# # regex_pattern_general=r'(?i)(roll)|()'
# regex_pattern=r'(?i)(?i)(\d+.?\d*\/?\d*.?\s?to\s?[a-z\s]{0,20}\d*.?\d*\/?\d*)|(seat.?height.?\s?\d+\"\s?\-?\s?\d*\s?\d*\/?\d*)|(\d\d\d?\d*\".{0,3}\d\d\.\d*\"\s?H)||()'              
# oc['matches']=oc['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# oc_na=oc[oc['matches'].astype(str)=='[]']
# oc=oc[oc['matches'].astype(str)!='[]']
# oc_na[curation_col]=r'n/a'

In [36]:
# len(oc_na) 

# Desk WorkBenches

In [37]:
# desk=height[height['buckets'].astype(str)=='Desks/Workstations/Workbenches']
# # regex_pattern_general=r'(?i)(roll)|()'
# regex_pattern=r'(?i)(?i)(height)|("H)|(\d\d\.?\d\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|()'              
# desk['matches']=desk['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# desk_na=desk[desk['matches'].astype(str)=='[]']
# desk_na[curation_col]=r'n/a'

In [38]:
# len(desk_na) 

# Chairs

In [47]:
# chair=height[height['buckets'].astype(str)=='Chairs']
# regex_pattern=r'(?i)(\d+\.?\s?\d*\/?\d*\"\s?h)|()'              
# chair['matches']=chair['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# chair_height=chair[chair['matches'].astype(str)!='[]']

In [48]:
# chair_three=chair[chair['matches'].astype(str)=='[]']
# regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'              
# chair_three['matches']=chair_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))

In [49]:
# chair_na=chair_three[chair_three['matches'].astype(str)=='[]']
# chair_na[curation_col]=r'n/a'

In [39]:
# print(len(chair_na)) 

# Drawers

In [40]:
drawer=height[height['buckets'].astype(str)=='Drawers']
print(len(drawer))
regex_pattern_drawer=r'(?i)(\d\d\.?\d*\s?h)|(\d+\s?\d*\/\d*\s?h)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|(\d+.?\d*\"\s?To\s?\d+")|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")|(height)|()'                           
drawer['matches']=drawer['long_desc'].apply(lambda x: re_extract(regex_pattern_drawer, x))
drawer_na=drawer[drawer['matches'].astype(str)=='[]']
drawer_na[curation_col]=r'n/a'
print(len(drawer_na))

268
0


In [41]:
drawer_next=drawer[drawer['matches'].astype(str)!='[]']
regex_pattern_short=r'(?i)(\d\d\.?\d*\s?h)|(\d+\s?\-?\d*\/\d*\s?h)|(\d+\.?\s?\-?\d*\/?\d*"H)|(height\s?without\s?casters.?\s?\d+\")|()'                           
drawer_next['new_matches']=drawer_next['long_desc'].apply(lambda x: re_extract(regex_pattern_short, x))
drawer_short=drawer_next[drawer_next['new_matches'].astype(str)!='[]']
print(len(drawer_short))

225


In [42]:
drawer_short['height']=drawer_short['new_matches'].str.get(0)
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_short['height']=drawer_short['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_short['height']=drawer_short['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
drawer_short['height'] = drawer_short['height'].apply(lambda x: remove_fraction(x))
drawer_short['height'] = drawer_short['height'].apply(lambda x: remove_frac(x))
drawer_short['height']=drawer_short['height'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_short['height']=drawer_short['height'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [43]:
drawer_short[curation_col]=drawer_short['height']
drawer_short[curation_col]=drawer_short[curation_col].apply(lambda x: str(x)+' in')

In [44]:
drawer_three=drawer_next[drawer_next['new_matches'].astype(str)=='[]']
regex_pattern_three=r'(?i)(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+\.\d*)|(\d+.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\s?\-?\d*\/\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+\.\d*)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*\/\d*.?\s?x\s?\d+.?\d*)|(\d+.?\s?x\s?\d+.?\s?x\s?\d+)|(\d+.?\d*\"\s?W\s?x\s?\d+.?\d*\/?\d*\"\s?D\s?x\s?\d+.?\d*\/?\d*\"\s?H)|(\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?x\s?\d+\.?\d*\"\s?)|()'
drawer_three['new_matches']=drawer_three['long_desc'].apply(lambda x: re_extract(regex_pattern_three, x))
drawer_three=drawer_three[drawer_three['new_matches'].astype(str)!='[]']

In [45]:
drawer_three['new']=drawer_three['new_matches'].str.get(0)
drawer_three[['length', 'width', 'height']]=drawer_three['new'].str.split("x",expand=True,)
drawer_three=drawer_three[(drawer_three['external_id'].astype(str)!='5637192984')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192983')&(drawer_three['external_id'].astype(str)!='5637192989')&(drawer_three['external_id'].astype(str)!='5637192990')]                      

In [46]:
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�','',str(measure)))
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r"\'|,|:",'',str(measure)))
drawer_three['height']=drawer_three['height'].apply(lambda measure: re.sub(r'\-',' ',str(measure)))
drawer_three['height']=drawer_three['height'].replace('"', '').apply(lambda quote: quote.replace('"', ''))

In [47]:
regex_pattern_new=r'(?i)(\d+.?\d*\/?\d*)|()'
drawer_three['height_new']=drawer_three['height'].apply(lambda x: re_extract(regex_pattern_new, x)).str.get(0)
drawer_three['height_new'] = drawer_three['height_new'].apply(lambda x: remove_fraction(x))

In [48]:
drawer_three['height_new']=drawer_three['height_new'].apply(lambda x: remove_fraction(x))
drawer_three['height_new']=drawer_three['height_new'].apply(lambda x: remove_frac(x))
drawer_three['height_new']=drawer_three['height_new'].astype(float)
TWOPLACES = Decimal(10) ** -2
drawer_three['height_new']=drawer_three['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
drawer_three[curation_col]=drawer_three['height_new']

In [49]:
drawer_three[curation_col]=drawer_three[curation_col].apply(lambda x:str(x)+' in')

In [64]:
print(len(drawer_short))

225


In [58]:
len(envelope)

5576

In [57]:
err

NameError: name 'err' is not defined

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

In [ ]:
len(envelope) print(len(drawer_three)) print(len(drawer_short))

# send to the folder for upload

In [18]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}-- envelope_na upload_Shawn.csv',index=False)

In [19]:
looks_good('Bed Bath & Beyond', attribute, df, envelope_na)

In [30]:
def looks_good_na(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets-NA - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [31]:
looks_good_na('Bed Bath & Beyond', attribute, df, total_na_values)

In [ ]:
matches_side_table+matches_stool+matches_tree